In [1]:
import pandas as pd
import numpy as np
import random
import time

# Metody doboru sąsiedztwa

In [2]:
def generate_neighbor(solution, neighbor_type='swap'):
    i, j = np.random.choice(len(solution), size=2, replace=False)
    neighbor = solution.copy()
    city1, city2 = neighbor[i], neighbor[j]
    city_pair = [city1, city2]
    if neighbor_type == 'swap':
        # Generuj sąsiada przez zamianę dwóch losowych miast
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    elif neighbor_type == 'insert':
        # Generuj sąsiada przez wstawienie losowego miasta w inne miejsce
        neighbor.insert(j, neighbor.pop(i))
    elif neighbor_type == 'reverse':
        # Generuj sąsiada przez odwrócenie kolejności miast pomiędzy dwoma losowymi indeksami
        if i > j:
            i, j = j, i
        neighbor[i:j + 1] = reversed(neighbor[i:j + 1])
    else:
        raise ValueError("Nieznany rodzaj sąsiedztwa: {}".format(neighbor_type))
    return neighbor, city_pair

# Funkcje trasy

In [3]:
# Stworzenie funkcji ustalenia początkowej trasy
def create_initial_route(distance_matrix):
    route = list(range(len(distance_matrix)))
    random.shuffle(route)
    return route

In [4]:
def calculate_total_distance(solution, distance_matrix):
    total_distance = 0
    num_cities = len(solution)

    for i in range(num_cities - 1):
        total_distance += distance_matrix.at[solution[i], solution[i + 1]]

    # Dodaj odległość z ostatniego do pierwszego miasta
    total_distance += distance_matrix.at[solution[num_cities - 1], solution[0]]

    return total_distance

# Tabu Search

In [5]:
def tabu_search(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type='swap', gen_neighbor_size=10):
    # Inicjalizacja bieżącego rozwiązania, najlepszego rozwiązania, listy tabu i najlepszego dystansu
    current_solution = initial_solution.copy()
    best_solution = current_solution.copy()
    tabu_list = []
    best_distance = calculate_total_distance(best_solution, distance_matrix)

    # Pętla główna algorytmu Tabu Search
    for _ in range(max_iterations):
        # Generowanie sąsiadów bieżącego rozwiązania
        neighbors = [generate_neighbor(current_solution, neighbor_type) for _ in range(gen_neighbor_size)]

        # Obliczanie dystansów dla kandydatów, którzy nie są na liście tabu
        candidate_distances = [(neighbor, city_pair, calculate_total_distance(neighbor, distance_matrix)) for neighbor, city_pair in neighbors if city_pair not in tabu_list]

        # Jeśli brak kandydatów spoza listy tabu, kontynuuj pętlę
        if not candidate_distances:
            continue

        # Wybieranie najlepszego kandydata
        candidate_solution, city_pair, candidate_distance = min(candidate_distances, key=lambda x: x[2])

        # Aktualizacja najlepszego rozwiązania, jeśli znaleziono lepsze
        if candidate_distance < best_distance:
            best_solution = candidate_solution
            best_distance = candidate_distance

        # Dodanie ruchu do listy tabu
        tabu_list.append(city_pair)
        if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)  # Usuwanie najstarszego ruchu z listy tabu

    # Zwracanie najlepszego rozwiązania i odpowiadającego mu dystansu
    return best_solution, best_distance


In [6]:
def tabu_search_with_time_limit(distance_matrix, initial_solution, tabu_list_size, max_time_seconds, neighbor_type='swap', gen_neighbor_size=10):
    # Inicjalizacja bieżącego rozwiązania, najlepszego rozwiązania, listy tabu i najlepszego dystansu
    current_solution = initial_solution.copy()
    best_solution = current_solution.copy()
    tabu_list = []
    best_distance = calculate_total_distance(best_solution, distance_matrix)

    # Początkowy czas wykonania algorytmu
    start_time = time.time()

    # Pętla główna algorytmu Tabu Search z ograniczeniem czasowym
    while (time.time() - start_time) < max_time_seconds:
        # Generowanie sąsiadów bieżącego rozwiązania
        neighbors = [generate_neighbor(current_solution, neighbor_type) for _ in range(gen_neighbor_size)]

        # Obliczanie dystansów dla kandydatów, którzy nie są na liście tabu
        candidate_distances = [(neighbor, city_pair, calculate_total_distance(neighbor, distance_matrix)) for neighbor, city_pair in neighbors if city_pair not in tabu_list]

        # Jeśli brak kandydatów spoza listy tabu, kontynuuj pętlę
        if not candidate_distances:
            continue

        # Wybieranie najlepszego kandydata
        candidate_solution, city_pair, candidate_distance = min(candidate_distances, key=lambda x: x[2])

        # Aktualizacja najlepszego rozwiązania, jeśli znaleziono lepsze
        if candidate_distance < best_distance:
            best_solution = candidate_solution
            best_distance = candidate_distance

        # Dodanie ruchu do listy tabu
        tabu_list.append(city_pair)
        if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)  # Usuwanie najstarszego ruchu z listy tabu

    # Zwracanie najlepszego rozwiązania i odpowiadającego mu dystansu
    return best_solution, best_distance


In [7]:
def tabu_search_multiple_runs(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type='swap', gen_neighbor_size=10, num_runs=5):
    best_overall_solution = initial_solution.copy()
    best_overall_distance = calculate_total_distance(best_overall_solution, distance_matrix)
    for _ in range(num_runs):
        # Wywołaj funkcję tabu_search i zapisz wyniki
        current_solution, current_distance = tabu_search(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type, gen_neighbor_size)

        # Sprawdź, czy bieżące rozwiązanie jest lepsze od najlepszego dotychczasowego
        if current_distance < best_overall_distance:
            best_overall_solution = current_solution
            best_overall_distance = current_distance

    # Wydrukuj informacje o najlepszym rozwiązaniu z wszystkich prób
    print("Rozmiar listy tabu: {}, maksymalna powtórzeń: {}, funkcja sąsiedztwa: {}, liczba tworzonych sąsiadów: {}".format(tabu_list_size, max_iterations, neighbor_type, gen_neighbor_size))
    print("Najlepsze rozwiązanie spośród {} prób: {}".format(num_runs, best_overall_solution))
    print("Długość trasy: {}".format(best_overall_distance))

In [8]:
def tabu_search_with_time_limit_multiple_runs(distance_matrix, initial_solution, tabu_list_size, max_time_seconds, neighbor_type='swap', gen_neighbor_size=10, num_runs=5):
    best_overall_solution = initial_solution.copy()
    best_overall_distance = calculate_total_distance(best_overall_solution, distance_matrix)

    for _ in range(num_runs):
        # Wywołaj funkcję tabu_search_with_time_limit i zapisz wyniki
        current_solution, current_distance = tabu_search_with_time_limit(distance_matrix, initial_solution, tabu_list_size, max_time_seconds, neighbor_type, gen_neighbor_size)

        # Sprawdź, czy bieżące rozwiązanie jest lepsze od najlepszego dotychczasowego
        if current_distance < best_overall_distance:
            best_overall_solution = current_solution
            best_overall_distance = current_distance

    # Wydrukuj informacje o najlepszym rozwiązaniu z wszystkich prób
    print("Rozmiar listy tabu: {}, maksymalny czas liczenia: {}, funkcja sąsiedztwa: {}, liczba tworzonych sąsiadów: {}".format(tabu_list_size, max_time_seconds, neighbor_type, gen_neighbor_size))
    print("Najlepsze rozwiązanie spośród {} prób: {}".format(num_runs, best_overall_solution))
    print("Długość trasy: {}".format(best_overall_distance))


## TSP_127

In [11]:
TSP_127 = pd.read_excel('Dane_TSP_127.xlsx', header=None)
# Rozwiazanie poczatkowe wygenerowane przez NN (Nearest Neighbor)
NN_127 = [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 64, 103]
# Długość trasy:  133970.64577239772

### Sąsiedztwo

In [10]:
tabu_search_multiple_runs(TSP_127, NN_127, 10, 10000, 'swap', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: swap, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [74, 83, 80, 125, 81, 82, 116, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 64, 103]
Długość trasy: 132981.67192517567


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: swap, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [74, 83, 80, 125, 81, 82, 116, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 64, 103]
Długość trasy: 132981.67192517567


In [11]:
tabu_search_multiple_runs(TSP_127, NN_127, 10, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


In [12]:
tabu_search_multiple_runs(TSP_127, NN_127, 10, 10000, 'reverse', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 103, 64, 98, 91, 88, 124, 58, 66, 73, 72, 67, 68, 69, 70]
Długość trasy: 131157.7299304231


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 103, 64, 98, 91, 88, 124, 58, 66, 73, 72, 67, 68, 69, 70]
Długość trasy: 131157.7299304231

### Długość listy tabu

In [13]:
tabu_search_multiple_runs(TSP_127, NN_127, 5, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 5, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 5, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353

In [14]:
tabu_search_multiple_runs(TSP_127, NN_127, 8, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 8, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 8, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353

In [15]:
tabu_search_multiple_runs(TSP_127, NN_127, 15, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 15, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 15, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353

In [16]:
tabu_search_multiple_runs(TSP_127, NN_127, 20, 10000, 'insert', 10, 5)

Rozmiar listy tabu: 20, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 20, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353

### Liczba iteracji

In [17]:
tabu_search_multiple_runs(TSP_127, NN_127, 10, 100, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 100, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 64, 103]
Długość trasy: 133297.45765344126


Rozmiar listy tabu: 10, maksymalna powtórzeń: 100, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 64, 103]
Długość trasy: 133297.45765344126

In [18]:
tabu_search_multiple_runs(TSP_127, NN_127, 10, 1000, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 1000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 10, maksymalna powtórzeń: 1000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353

In [19]:
tabu_search_multiple_runs(TSP_127, NN_127, 10, 10000,'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


In [20]:
tabu_search_multiple_runs(TSP_127, NN_127, 10, 100000, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 100000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 10, maksymalna powtórzeń: 100000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353

### Czas liczenia

In [21]:
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_127, 10, 10, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 10, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 10, maksymalny czas liczenia: 10, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


In [22]:
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_127, 10, 30, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 30, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 10, maksymalny czas liczenia: 30, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353

In [23]:
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_127, 10, 60, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 60, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 10, maksymalny czas liczenia: 60, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353

In [24]:
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_127, 10, 90, 'insert', 10, 5)

Rozmiar listy tabu: 10, maksymalny czas liczenia: 90, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 10, maksymalny czas liczenia: 90, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 10
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353

### Liczba wygenerowanych sąsiedztw

In [25]:
tabu_search_multiple_runs(TSP_127, NN_127, 10, 10000,'insert', 5, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 5
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 5
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353

In [26]:
tabu_search_multiple_runs(TSP_127, NN_127, 10, 10000,'insert', 8, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 8
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 8
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353

In [27]:
tabu_search_multiple_runs(TSP_127, NN_127, 10, 10000,'insert', 15, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 15
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 15
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


In [28]:
tabu_search_multiple_runs(TSP_127, NN_127, 10, 10000,'insert', 20, 5)

Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 20
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353


Rozmiar listy tabu: 10, maksymalna powtórzeń: 10000, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 20
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 70, 69, 68, 67, 72, 73, 66, 58, 124, 88, 91, 98, 103]
Długość trasy: 132806.68125180353

### Wnioski po testach: 
Przeprowadzone testy wskazują, że poza sąsiedztwem lub bardzo małą ilością iteracji, dobór parametrów nie ma większego znaczenia, jeżeli mamy już rozwiązanie w okolicy optymalnego. W testach celowo została użyta funkcja insert doboru sąsiedztwa ze względu na jej przeciętne wyniki, co pozwala na zbadanie wpływu innych parametrów. 

### Poszukiwanie najlepszego rozwiązania dla TSP_127
Do poszukania najlepszego rozwiązanie wykorzystamy trasę wyznaczoną przez algorytm NN oraz parametry, które najlepiej sprawdziły się w wyznaczaniu trasy bazującącej na losowym roziązaniu.

In [12]:
NN_improved = [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 103, 64, 98, 91, 88, 124, 58, 66, 73, 72, 67, 68, 69, 70]
# Długość trasy: 131157.7299304231
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_improved, 15, 45,'insert', 30, 5)

Rozmiar listy tabu: 15, maksymalny czas liczenia: 45, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 30
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 68, 69, 70]
Długość trasy: 129993.76540982892


In [13]:
NN_improved_2 = [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 86, 85, 84, 87, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 68, 69, 70]
# Długość trasy: 129993.76540982892
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_improved_2, 15, 30,'insert', 30, 5)

Rozmiar listy tabu: 15, maksymalny czas liczenia: 30, funkcja sąsiedztwa: insert, liczba tworzonych sąsiadów: 30
Najlepsze rozwiązanie spośród 5 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 68, 69, 70]
Długość trasy: 129320.57729087246


In [14]:
NN_improved_3 = [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 68, 69, 70]
# Długość trasy: 129320.57729087246
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_improved_3, 15, 30,'reverse', 30, 10)

Rozmiar listy tabu: 15, maksymalny czas liczenia: 30, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 30
Najlepsze rozwiązanie spośród 10 prób: [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 70, 69, 68]
Długość trasy: 127913.21838455701


In [15]:
NN_improved_4 = [116, 83, 80, 125, 81, 82, 74, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 70, 69, 68]
# Długość trasy: 127913.21838455701
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_improved_4, 15, 30,'reverse', 30, 10)

Rozmiar listy tabu: 15, maksymalny czas liczenia: 30, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 30
Najlepsze rozwiązanie spośród 10 prób: [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 70, 69, 68]
Długość trasy: 126509.53882319551


In [16]:
NN_improved_5 = [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 56, 53, 44, 102, 43, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 70, 69, 68]
# Długość trasy: 126509.53882319551
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_improved_5, 15, 30,'reverse', 30, 10)

Rozmiar listy tabu: 15, maksymalny czas liczenia: 30, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 30
Najlepsze rozwiązanie spośród 10 prób: [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 39, 41, 31, 28, 27, 121, 32, 24, 25, 37, 38, 33, 42, 29, 26, 30, 11, 13, 40, 36, 35, 34, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 70, 69, 68]
Długość trasy: 125941.84025533627


In [17]:
NN_improved_6 = [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 39, 41, 31, 28, 27, 121, 32, 24, 25, 37, 38, 33, 42, 29, 26, 30, 11, 13, 40, 36, 35, 34, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 70, 69, 68]
# Długość trasy: 125941.84025533627
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_improved_6, 15, 30,'reverse', 30, 10)

Rozmiar listy tabu: 15, maksymalny czas liczenia: 30, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 30
Najlepsze rozwiązanie spośród 10 prób: [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 70, 69, 68]
Długość trasy: 125395.7582403361


In [18]:
NN_improved_7 = [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 21, 3, 22, 23, 5, 105, 14, 107, 19, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 70, 69, 68]
# Długość trasy: 125395.7582403361
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_improved_7, 15, 60,'reverse', 60, 10)

Rozmiar listy tabu: 15, maksymalny czas liczenia: 60, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 60
Najlepsze rozwiązanie spośród 10 prób: [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 70, 69, 68]
Długość trasy: 125116.47190141668


In [19]:
NN_improved_8 = [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 121, 27, 28, 31, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 70, 69, 68]
# Długość trasy: 125116.47190141668
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_improved_8, 15, 60,'reverse', 60, 10)

Rozmiar listy tabu: 15, maksymalny czas liczenia: 60, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 60
Najlepsze rozwiązanie spośród 10 prób: [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 31, 28, 27, 121, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 70, 69, 68]
Długość trasy: 124869.34075640207


In [20]:

NN_improved_9 = [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 31, 28, 27, 121, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 73, 72, 67, 70, 69, 68]
# Długość trasy: 124869.34075640207
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_improved_9, 15, 30,'reverse', 60, 5)

Rozmiar listy tabu: 15, maksymalny czas liczenia: 30, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 60
Najlepsze rozwiązanie spośród 5 prób: [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 31, 28, 27, 121, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 72, 73, 67, 70, 69, 68]
Długość trasy: 124640.58315380245


In [21]:
NN_improved_10 = [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 31, 28, 27, 121, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 72, 73, 67, 70, 69, 68]
# Długość trasy: 124640.58315380245
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_improved_10, 15, 30,'reverse', 60, 10)

Rozmiar listy tabu: 15, maksymalny czas liczenia: 30, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 60
Najlepsze rozwiązanie spośród 10 prób: [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 31, 28, 121, 27, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 72, 73, 67, 70, 69, 68]
Długość trasy: 124471.73758395515


In [22]:
NN_improved_11 = [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 31, 28, 121, 27, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 9, 119, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 72, 73, 67, 70, 69, 68]
# Długość trasy: 124471.73758395515
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_improved_11, 15, 30,'reverse', 60, 10)

Rozmiar listy tabu: 15, maksymalny czas liczenia: 30, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 60
Najlepsze rozwiązanie spośród 10 prób: [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 31, 28, 121, 27, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 119, 9, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 72, 73, 67, 70, 69, 68]
Długość trasy: 124316.66035361464


In [23]:
NN_improved_12 = [74, 82, 81, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 31, 28, 121, 27, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 119, 9, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 72, 73, 67, 70, 69, 68]
# Długość trasy: 124316.66035361464

tabu_search_with_time_limit_multiple_runs(TSP_127, NN_improved_12, 15, 60,'reverse', 60, 10)

Rozmiar listy tabu: 15, maksymalny czas liczenia: 60, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 60
Najlepsze rozwiązanie spośród 10 prób: [74, 81, 82, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 31, 28, 121, 27, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 119, 9, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 72, 73, 67, 70, 69, 68]
Długość trasy: 124234.51167135675


In [24]:
NN_improved_13 = [74, 81, 82, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 31, 28, 121, 27, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 119, 9, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 117, 47, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 72, 73, 67, 70, 69, 68]
# Długość trasy: 124234.51167135675
tabu_search_with_time_limit_multiple_runs(TSP_127, NN_improved_13, 15, 60,'reverse', 60, 10)

Rozmiar listy tabu: 15, maksymalny czas liczenia: 60, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 60
Najlepsze rozwiązanie spośród 10 prób: [74, 81, 82, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 31, 28, 121, 27, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 119, 9, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 47, 117, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 72, 73, 67, 70, 69, 68]
Długość trasy: 124155.11931181168


Wnioski: Algorytm Tabu_Search ma problem ze znalezieniem optimum lokalnego dla dużych zbiorów danych. Wymaga wielokrotnego "restartu" z trasą lepszej jakości uzyskaną w poprzedniej iteracji. Po 13 iteracjach nadal jest w stanie poprawić swój wynik, jednak zysk z tej poprawy maleje, można to zobaczyć na wykresie poprawy wyniku w pliku głównym.

### Ostateczne poszukiwanie rozwiązania - funkcja rekurecnycjna

In [10]:
# Funkcja rekurencyjnie wykorzystująca tabu search do znalezienia najlepszego rozwiązania
def tabu_search_recursive(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type='swap', gen_neighbor_size=10, num_runs=5):
    best_overall_solution = initial_solution.copy()
    best_overall_distance = calculate_total_distance(best_overall_solution, distance_matrix)
    for _ in range(num_runs):
        # Wywołaj funkcję tabu_search i zapisz wyniki
        current_solution, current_distance = tabu_search(distance_matrix, initial_solution, tabu_list_size, max_iterations, neighbor_type, gen_neighbor_size)

        # Sprawdź, czy bieżące rozwiązanie jest lepsze od najlepszego dotychczasowego
        if current_distance < best_overall_distance:
            best_overall_solution = current_solution
            best_overall_distance = current_distance

    # Wydrukuj informacje o najlepszym rozwiązaniu z wszystkich prób
    print("Rozmiar listy tabu: {}, maksymalna powtórzeń: {}, funkcja sąsiedztwa: {}, liczba tworzonych sąsiadów: {}".format(tabu_list_size, max_iterations, neighbor_type, gen_neighbor_size))
    print("Najlepsze rozwiązanie spośród {} prób: {}".format(num_runs, best_overall_solution))
    print("Długość trasy: {}".format(best_overall_distance))

    # Jeśli znaleziono lepsze rozwiązanie, wywołaj funkcję rekurencyjnie
    if best_overall_solution != initial_solution:
        tabu_search_recursive(distance_matrix, best_overall_solution, tabu_list_size, max_iterations, neighbor_type, gen_neighbor_size, num_runs)

In [12]:
NN_improved_14 = [74, 81, 82, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 35, 36, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 31, 28, 121, 27, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 119, 9, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 47, 117, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 72, 73, 67, 70, 69, 68]
# Długość trasy: 124155.11931181168
tabu_search_recursive(TSP_127, NN_improved_14, 15, 3000, 'reverse', 60, 10)

Rozmiar listy tabu: 15, maksymalna powtórzeń: 3000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 60
Najlepsze rozwiązanie spośród 10 prób: [74, 81, 82, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 36, 35, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 31, 28, 121, 27, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 12, 114, 119, 9, 2, 89, 115, 59, 61, 60, 90, 57, 63, 99, 112, 64, 65, 54, 46, 48, 52, 47, 117, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 72, 73, 67, 70, 69, 68]
Długość trasy: 124095.08300719385
Rozmiar listy tabu: 15, maksymalna powtórzeń: 3000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 60
Najlepsze rozwiązanie spośród 10 prób: [74, 81, 82, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1

Wnioski: Funkcja rekurencyjna po dodatkowych 5 powtórzeniach znalazła optymalny wynik, oto on:

Rozmiar listy tabu: 15, maksymalna powtórzeń: 3000, funkcja sąsiedztwa: reverse, liczba tworzonych sąsiadów: 60
Najlepsze rozwiązanie spośród 10 prób: [74, 81, 82, 125, 80, 83, 116, 75, 77, 79, 78, 76, 17, 20, 16, 19, 107, 14, 105, 5, 23, 22, 3, 21, 18, 71, 7, 8, 10, 113, 104, 6, 0, 15, 1, 50, 34, 36, 35, 40, 13, 11, 30, 26, 29, 42, 33, 38, 37, 25, 24, 32, 28, 31, 121, 27, 41, 39, 43, 102, 44, 53, 56, 120, 4, 55, 123, 51, 49, 114, 12, 119, 9, 2, 89, 115, 59, 61, 60, 90, 99, 57, 63, 112, 64, 65, 54, 46, 48, 52, 47, 117, 45, 93, 111, 110, 106, 126, 92, 94, 122, 96, 97, 100, 101, 62, 118, 95, 108, 87, 86, 85, 84, 109, 103, 98, 91, 88, 124, 58, 66, 72, 73, 67, 70, 69, 68]
Długość trasy: 123789.3814391673